#### 1) Distribution setup

Discretize normal distribution (with mean = 0.15, standard deviation = 0.20) into $2^n$ bins.
- `grid_points`: bin midpoints (representative return value per bin)
- `edges`: bin edges
- `probs`: probability mass per bin (sums to 1)
Because we truncate to $\pm k\sigma$ and renormalize, this introduces **discretization/modeling error**.

In [ ]:
# Discretize normal distribution into 2^n bins

def build_normal_return_grid_probs(mu=0.15, sigma=0.20, n_qubits=7, k_sigmas=4.0):
    """Discretize a normal distribution into 2^n_qubits bins over [mu-kσ, mu+kσ]."""
    N = 2 ** n_qubits
    r_min = mu - k_sigmas * sigma
    r_max = mu + k_sigmas * sigma

    edges = np.linspace(r_min, r_max, N + 1)
    grid_centers = 0.5 * (edges[:-1] + edges[1:])

    nd = NormalDist(mu=mu, sigma=sigma)
    cdf = np.array([nd.cdf(x) for x in edges])
    probs = np.diff(cdf)

    # numerical safety + renormalize (this makes the distribution truncated to [r_min, r_max])
    # introduce modeling error
    probs = np.maximum(probs, 0.0)
    probs = probs / probs.sum()

    return grid_centers, probs, edges, (mu, sigma, r_min, r_max)

# Parameters (edit to explore sensitivity later)
MU = 0.15
SIGMA = 0.20
NUM_QUBITS = 7
K_SIGMAS = 4.0

grid_points, probs, edges, (mu, sigma, r_min, r_max) = build_normal_return_grid_probs(
    mu=MU, sigma=SIGMA, n_qubits=NUM_QUBITS, k_sigmas=K_SIGMAS
)

print(f"Grid size: {len(probs)} bins (n_qubits={NUM_QUBITS})")
print(f"Truncation range: [{r_min:.3f}, {r_max:.3f}]")

# Visualize the discretized distribution
plt.figure(figsize=(9,3))
plt.plot(grid_points, probs)
plt.title("Discretized Gaussian Return Distribution (bin probabilities)")
plt.xlabel("Return")
plt.ylabel("Probability mass per bin")
plt.grid(True, alpha=0.3)
plt.show()


#### 2) VaR definition and ground-truth values

For a left-tail VaR at confidence level **(1-α)**, we use α as the tail probability (e.g., α=0.05 for 95% VaR):

- Tail event: **Return ≤ threshold**
- Tail probability: **P(Return ≤ threshold) = α**
- VaR (return threshold): **quantile at level α**

We compute:
- `VaR_cont`: continuous normal quantile (untruncated model)
- `VaR_disc`: exact VaR under the discretized/truncated distribution (`probs`) — this is what the quantum circuit actually encodes

In [ ]:


ALPHA = 0.05  # 5% tail -> 95% VaR confidence level

def var_continuous_normal(alpha, mu=MU, sigma=SIGMA):
    return norm.ppf(alpha, loc=mu, scale=sigma)

def cdf_discrete_at_index(index, probs):
    return float(np.sum(probs[:index]))

def var_discrete_from_probs(alpha, grid_points, probs):
    """Return the smallest index i such that CDF(i) >= alpha."""
    cdf = np.cumsum(probs)
    idx = int(np.searchsorted(cdf, alpha, side='left'))
    idx = np.clip(idx, 0, len(grid_points)-1)
    return idx, float(grid_points[idx])

VaR_cont = var_continuous_normal(ALPHA, mu=mu, sigma=sigma)

idx_disc, VaR_disc = var_discrete_from_probs(ALPHA, grid_points, probs)

print(f"ALPHA (tail prob): {ALPHA}")
print(f"Continuous Normal VaR (untruncated): {VaR_cont:.6f}")
print(f"Discrete/Truncated VaR (encoded in probs): index={idx_disc}, VaR_disc={VaR_disc:.6f}")
print(f"Discrete CDF at VaR_disc index: {cdf_discrete_at_index(idx_disc, probs):.6f}")  # should be ~alpha


#### 3) Quantum encoding, threshold oracle, and IQAE tail probability estimation

Set up discretized normal distribution into quantum states.
- `asset` register in amplitudes proportional to `probs`
- indicator qubit `ind` flipped when **asset index < threshold_index**
$$|\psi>=\sum_{i<\text{thres}}^{N}\sqrt{p_i}|i>+\sum_{i\ge\text{thres}}^{N}\sqrt{p_i}|i>

In [ ]:
# --- Quantum state preparation and oracle ---
# GLOBAL_INDEX is a mutable threshold used by the oracle.
GLOBAL_INDEX = 0

@qfunc(synthesize_separately=True)
def state_preparation(asset: QArray[QBit], ind: QBit):
    load_distribution(asset=asset)
    payoff(asset=asset, ind=ind)

@qfunc
def load_distribution(asset: QNum):
    inplace_prepare_state(probs, bound=0, target=asset)

@qperm
def payoff(asset: Const[QNum], ind: QBit):
    # Tail event: asset index < GLOBAL_INDEX  (left tail)
    ind ^= asset < GLOBAL_INDEX

def get_iqae_query_count(iqae_res):
    """Try to extract oracle/query counts from the result object (SDK-dependent)."""
    # common names used across libraries
    for name in [
        'num_oracle_queries', 'oracle_queries', 'n_oracle_queries',
        'num_queries', 'queries', 'query_count',
        'num_grover_calls', 'grover_calls', 'num_iterations',
        'shots', 'num_shots'
    ]:
        if hasattr(iqae_res, name):
            try:
                return int(getattr(iqae_res, name))
            except Exception:
                pass
    return None

def iqae_tail_prob(threshold_index, epsilon=0.05, alpha_ci=0.01):
    """Estimate tail probability P(asset < threshold_index) using IQAE."""
    global GLOBAL_INDEX
    GLOBAL_INDEX = int(threshold_index)

    iqae = IQAE(
        state_prep_op=state_preparation,
        problem_vars_size=NUM_QUBITS,
        constraints=Constraints(max_width=28),
        preferences=Preferences(machine_precision=NUM_QUBITS),
    )
    _ = iqae.get_qprog()  # ensures synthesis
    res = iqae.run(epsilon=float(epsilon), alpha=float(alpha_ci))

    est = float(res.estimation)
    ci = tuple(res.confidence_interval) if hasattr(res, 'confidence_interval') else None
    qcount = get_iqae_query_count(res)

    return est, ci, qcount

# Quick sanity check at the discrete VaR index:
p_true = cdf_discrete_at_index(idx_disc, probs)
p_hat, ci, qcount = iqae_tail_prob(idx_disc, epsilon=0.05, alpha_ci=0.01)
print(f"Tail prob at idx={idx_disc}: true={p_true:.4f}, IQAE={p_hat:.4f}, CI={ci}, queries={qcount}")

#### 4) Quantum VaR via bisection search

We search for the smallest index `i` such that the estimated tail probability satisfies:

- `P(Return ≤ threshold_i) ≈ α`

We use IQAE to estimate the tail probability at each candidate threshold.

In [ ]:
def quantum_var_bisection(alpha, probs, epsilon=0.05, alpha_ci=0.01, max_iters=25):
    """Find discrete VaR index using bisection + IQAE tail probability estimates."""
    lo, hi = 0, len(probs) - 1
    best = hi

    for it in range(max_iters):
        mid = (lo + hi) // 2
        p_hat, ci, qcount = iqae_tail_prob(mid, epsilon=epsilon, alpha_ci=alpha_ci)

        # Bisection logic: want CDF(mid) >= alpha
        if p_hat >= alpha:
            best = mid
            hi = mid - 1
        else:
            lo = mid + 1

        if lo > hi:
            break

    return best, float(grid_points[best])

# Run quantum VaR
EPSILON_IQAE = 0.05
ALPHA_CI = 0.01

idx_q, VaR_q = quantum_var_bisection(ALPHA, probs, epsilon=EPSILON_IQAE, alpha_ci=ALPHA_CI, max_iters=25)

print(f"Quantum VaR (discrete index): {idx_q}")
print(f"Quantum VaR (return value):   {VaR_q:.6f}")
print(f"Discrete truth VaR_disc:      {VaR_disc:.6f}")
print(f"Estimation error |VaR_q - VaR_disc| = {abs(VaR_q - VaR_disc):.6e}")